In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, datetime,timedelta
import yfinance as yf
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mdates
import scipy.optimize
import pyodbc

In [20]:
conn = pyodbc.connect('Driver={MySQL ODBC 8.0 ANSI Driver};'
                      'Server=localhost;'
                      'Database=bolsa_valores;'
                      'UID=root;'
                      'PASSWORD=1234;'
                      'PORT=3306;')
cursor = conn.cursor()

InterfaceError: ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Nome da fonte de dados não encontrado e nenhum driver padrão especificado (0) (SQLDriverConnect)')

In [17]:
chamada_api = yf.Ticker('PETR4.SA').history(period='36mo')
df_hist = pd.DataFrame(chamada_api).reset_index()
df_hist = df_hist[["Date","Open","High","Low","Close","Volume","Dividends"]]
df_hist




,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2020-03-20 00:00:00-03:00,6.226772,6.412008,5.618819,5.699563,149029900,0.0,0.0
1,2020-03-23 00:00:00-03:00,5.751808,5.785056,5.357588,5.462080,107092400,0.0,0.0
2,2020-03-24 00:00:00-03:00,5.970291,6.431006,5.832552,6.293266,114259200,0.0,0.0
3,2020-03-25 00:00:00-03:00,6.288517,7.024710,6.093782,6.801477,158286200,0.0,0.0
4,2020-03-26 00:00:00-03:00,6.839474,7.214695,6.559246,6.834725,130618400,0.0,0.0
...,...,...,...,...,...,...,...,...
740,2023-03-14 00:00:00-03:00,24.280001,24.730000,23.600000,23.760000,58916700,0.0,0.0
741,2023-03-15 00:00:00-03:00,23.330000,23.480000,22.799999,23.340000,93708800,0.0,0.0
742,2023-03-16 00:00:00-03:00,23.400000,23.520000,22.969999,23.260000,66821000,0.0,0.0
743,2023-03-17 00:00:00-03:00,23.100000,23.510000,22.860001,23.510000,93502700,0.0,0.0


In [ ]:
symbol_list_ibrx = list(pd.read_csv('ibrx.csv',delimiter=";").index)
df_final = pd.DataFrame()
for ativo in symbol_list_ibrx:
    chamada_api = yf.Ticker(ativo + ".SA").history(period='1mo')
    df_hist = pd.DataFrame(chamada_api).reset_index()
    df_hist["Symbol"] = ativo
    df_hist = df_hist[["Symbol","Date","Open","High","Low","Close","Volume","Dividends"]]
    df_final = pd.concat([df_hist, df_final])
    query = "INSERT INTO nome_da_tabela (coluna1, coluna2, coluna3) VALUES (?, ?, ?)"
    valores = ("valor1", "valor2", "valor3")
    cursor.execute(query, valores)
    conn.commit()
df_final
conn.close()